In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [13]:

# Load data

# Load precomputed feature matrix
X = pd.read_csv("./VMMR_subset_images/train_features.csv", header=None).values

# Load labels (assumes format like: label, filename)
df_labels = pd.read_csv("./VMMR_subset_images/train_data.csv", header=None, names=["label", "filename"])

# X_train = pd.read_csv(X_train_path)
# y_train = pd.read_csv(y_train_path).values.ravel()

# X_train = X_train.values




In [15]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(df_labels["label"])

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = MLPClassifier(hidden_layer_sizes=(128, 64), activation='relu', max_iter=300)
model.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=300)

In [29]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"\nAccuracy: {accuracy:.4f}\n")

print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Accuracy: 0.8000

Classification Report:
                               precision    recall  f1-score   support

   chevrolet_impala_2004-2005       0.79      0.79      0.79       107
dodge_grand caravan_2005-2006       0.85      0.85      0.85        96
      ford_explorer_2001-2002       0.91      0.92      0.92       113
       ford_mustang_2000-2001       0.80      0.87      0.84       102
        ford_taurus_2002-2003       0.87      0.86      0.87        88
       honda_accord_2005-2006       0.57      0.66      0.61        90
      honda_odyssey_2005-2006       0.82      0.84      0.83        89
      nissan_altima_2005-2006       0.83      0.81      0.82       106
       toyota_camry_2002-2003       0.73      0.68      0.70       100
   volkswagen_jetta_2002-2003       0.83      0.70      0.76       109

                     accuracy                           0.80      1000
                    macro avg       0.80      0.80      0.80      1000
                 weighted avg    

In [ ]:
# MLP Evaluating Everything

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
import warnings
warnings.filterwarnings("ignore")

# ----------------------------
# Load Data
# ----------------------------

X = pd.read_csv("./VMMR_subset_images/train_features.csv", header=None).values
df_labels = pd.read_csv("./VMMR_subset_images/train_data.csv", header=None, names=["label", "filename"])

le = LabelEncoder()
y = le.fit_transform(df_labels["label"])

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# ----------------------------
# Parameter Options
# ----------------------------

scalers = {
    "None": None,
    "StandardScaler": StandardScaler(),
    "MinMaxScaler": MinMaxScaler()
}

pca_options = [None, 50, 100, 150]

feature_selectors = {
    "None": None,
    "SelectKBest": SelectKBest(score_func=f_classif, k=50),
    "SelectKBest_100": SelectKBest(score_func=f_classif, k=100),
    "SelectKBest_150": SelectKBest(score_func=f_classif, k=150),
}

classifiers = {
    "MLP_50": lambda: MLPClassifier(hidden_layer_sizes=(50,), max_iter=1000, early_stopping=True, random_state=42),
    "MLP_100_50": lambda: MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, early_stopping=True, random_state=42),
    "RandomForest": lambda: RandomForestClassifier(n_estimators=100, random_state=42)
}

# ----------------------------
# Cross-validation Setup
# ----------------------------

skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)

results = []

# ----------------------------
# Grid Search Loop
# ----------------------------

for scaler_name, scaler in scalers.items():
    for pca_n in pca_options:
        for selector_name, selector in feature_selectors.items():
            for clf_name, clf_func in classifiers.items():

                val_scores, f1_scores, auc_scores = [], [], []

                for train_idx, val_idx in skf.split(X_train, y_train):
                    X_tr, X_val = X_train[train_idx], X_train[val_idx]
                    y_tr, y_val = y_train[train_idx], y_train[val_idx]

                    # Scaling
                    if scaler:
                        fitted_scaler = scaler.fit(X_tr)
                        X_tr = fitted_scaler.transform(X_tr)
                        X_val = fitted_scaler.transform(X_val)

                    # Feature Selection
                    if selector:
                        fitted_selector = selector.fit(X_tr, y_tr)
                        X_tr = fitted_selector.transform(X_tr)
                        X_val = fitted_selector.transform(X_val)

                    # PCA
                    if pca_n:
                        pca = PCA(n_components=pca_n)
                        X_tr = pca.fit_transform(X_tr)
                        X_val = pca.transform(X_val)

                    clf = clf_func()
                    clf.fit(X_tr, y_tr)
                    preds = clf.predict(X_val)

                    val_scores.append(accuracy_score(y_val, preds))
                    f1_scores.append(f1_score(y_val, preds, average='weighted'))

                    try:
                        auc_scores.append(roc_auc_score(
                            pd.get_dummies(y_val), pd.get_dummies(preds), multi_class='ovr'))
                    except:
                        auc_scores.append(0)  # fallback if AUC fails

                    
                    if len(val_scores) == 1 and val_scores[0] < 0.6:
                        print(f"Skipping {scaler_name}, PCA={pca_n}, FS={selector_name}, Clf={clf_name} (Acc: {val_scores[0]:.2f})")
                        break

                results.append({
                    "Scaler": scaler_name,
                    "PCA": pca_n,
                    "FeatureSelector": selector_name,
                    "Classifier": clf_name,
                    "Accuracy": np.mean(val_scores),
                    "F1": np.mean(f1_scores),
                    "AUC": np.mean(auc_scores)
                })

# ----------------------------
# Output Results
# ----------------------------

results_df = pd.DataFrame(results)
print("\nTop Configurations by Accuracy:")
print(results_df.sort_values(by="Accuracy", ascending=False).head())

print("\nTop Configurations by F1:")
print(results_df.sort_values(by="F1", ascending=False).head())

print("\nTop Configurations by AUC:")
print(results_df.sort_values(by="AUC", ascending=False).head())

In [16]:
# MLP Best

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import warnings
warnings.filterwarnings("ignore")

# ----------------------------
# Load and Prepare Data
# ----------------------------

X = pd.read_csv("./VMMR_subset_images/train_features.csv", header=None).values
df_labels = pd.read_csv("./VMMR_subset_images/train_data.csv", header=None, names=["label", "filename"])

le = LabelEncoder()
y = le.fit_transform(df_labels["label"])

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# ----------------------------
# Best Preprocessing Setup
# ----------------------------

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# ----------------------------
# Tuned Classifier Variants
# ----------------------------

classifiers = {
    "MLP_50": MLPClassifier(hidden_layer_sizes=(50,), alpha=0.0005, max_iter=500, early_stopping=True, random_state=42),
    "MLP_75": MLPClassifier(hidden_layer_sizes=(75,), alpha=0.0005, max_iter=500, early_stopping=True, random_state=42),
    "MLP_50_50": MLPClassifier(hidden_layer_sizes=(50, 50), alpha=0.0005, max_iter=500, early_stopping=True, random_state=42),
    "MLP_100_25": MLPClassifier(hidden_layer_sizes=(100, 25), alpha=0.0005, max_iter=500, early_stopping=True, random_state=42)
}

# ----------------------------
# Cross-Validation Setup
# ----------------------------

skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
results = []

# ----------------------------
# Evaluation Loop
# ----------------------------

for name, clf in classifiers.items():
    acc_scores, f1_scores, auc_scores = [], [], []

    for train_idx, val_idx in skf.split(X_train, y_train):
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]

        clf.fit(X_tr, y_tr)
        preds = clf.predict(X_val)

        acc_scores.append(accuracy_score(y_val, preds))
        f1_scores.append(f1_score(y_val, preds, average='weighted'))
        try:
            auc_scores.append(roc_auc_score(pd.get_dummies(y_val), pd.get_dummies(preds), multi_class='ovr'))
        except:
            auc_scores.append(0)

    results.append({
        "Classifier": name,
        "Accuracy": np.mean(acc_scores),
        "F1": np.mean(f1_scores),
        "AUC": np.mean(auc_scores)
    })

# ----------------------------
# Output Results
# ----------------------------

results_df = pd.DataFrame(results)
print("\nTop Tuned MLP Configurations:")
print(results_df.sort_values(by="Accuracy", ascending=False).head())

# ----------------------------
# Final Training & Testing
# ----------------------------

# Rebuild and retrain best model on full training data
final_model = MLPClassifier(hidden_layer_sizes=(75,), alpha=0.0005, max_iter=500,
                            early_stopping=True, random_state=42)
final_model.fit(X_train, y_train)

# Predict on test set
y_pred = final_model.predict(X_test)

# Evaluate
from sklearn.metrics import classification_report, confusion_matrix

print("\nFinal Evaluation on Test Set:")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred, average='weighted'))
print("AUC Score:", roc_auc_score(pd.get_dummies(y_test), pd.get_dummies(y_pred), multi_class='ovr'))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))



Top Tuned MLP Configurations:
   Classifier  Accuracy        F1       AUC
1      MLP_75   0.76425  0.764171  0.869028
0      MLP_50   0.75500  0.755658  0.863889
3  MLP_100_25   0.74850  0.748071  0.860278
2   MLP_50_50   0.73650  0.736039  0.853611

Final Evaluation on Test Set:
Accuracy: 0.802
F1 Score: 0.8024491123177213
AUC Score: 0.8900000000000002

Classification Report:
                               precision    recall  f1-score   support

   chevrolet_impala_2004-2005       0.81      0.78      0.80       100
dodge_grand caravan_2005-2006       0.85      0.89      0.87       100
      ford_explorer_2001-2002       0.92      0.93      0.93       100
       ford_mustang_2000-2001       0.82      0.85      0.83       100
        ford_taurus_2002-2003       0.83      0.79      0.81       100
       honda_accord_2005-2006       0.63      0.66      0.64       100
      honda_odyssey_2005-2006       0.89      0.85      0.87       100
      nissan_altima_2005-2006       0.73      0.83

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from torch.utils.data import Dataset
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image


# -----------------------------
# CONFIG
# -----------------------------

image_dir = "./VMMR_subset_images/train_images/"  # path to all class subfolders
csv_file = "./VMMR_subset_images/train_data.csv"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

batch_size = 32
learning_rates = [0.001, 0.01]
dropouts = [0.3, 0.5]
conv_configs = [(16, 32), (32, 64)]  # (conv1 filters, conv2 filters)

# -----------------------------
# DATASET
# -----------------------------


# Load your CSV
df = pd.read_csv(csv_file, header=None, names=["label", "filename"])

# Add full relative path
df["path"] = df["label"] + "/" + df["filename"]

# Encode the labels
le = LabelEncoder()
df["target"] = le.fit_transform(df["label"])

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

class VehicleImageDataset(Dataset):
    def __init__(self, df, root_dir, transform=None):
        self.df = df
        self.root_dir = root_dir  # e.g., "./train_images"
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.df.iloc[idx]["path"])
        if not os.path.exists(img_path):
            raise FileNotFoundError(f"Missing file: {img_path}")
        image = Image.open(img_path).convert("RGB")
        label = self.df.iloc[idx]["target"]

        if self.transform:
            image = self.transform(image)

        return image, label


# -----------------------------
# MODEL
# -----------------------------

class SimpleCNN(nn.Module):
    def __init__(self, conv1_out, conv2_out, dropout_rate, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(3, conv1_out, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(conv1_out, conv2_out, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(conv2_out * 16 * 16, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# -----------------------------
# CROSS-VALIDATION
# -----------------------------
dataset = VehicleImageDataset(df, root_dir=image_dir, transform=transform)
skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
best_score = 0
results = []

X = np.arange(len(df))
y = df["target"].values

print("\nRunning CNN Cross-Validation...\n")

for conv1_out, conv2_out in conv_configs:
    for dropout in dropouts:
        for lr in learning_rates:
            fold_scores = []

            for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
                train_subset = Subset(dataset, train_idx)
                val_subset = Subset(dataset, val_idx)

                train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
                val_loader = DataLoader(val_subset, batch_size=batch_size)

                model = SimpleCNN(conv1_out, conv2_out, dropout, len(le.classes_)).to(device)
                optimizer = torch.optim.Adam(model.parameters(), lr=lr)
                criterion = nn.CrossEntropyLoss()

                # Train
                for epoch in range(5):  # Keep small for CV speed
                    model.train()
                    for images, labels in train_loader:
                        images, labels = images.to(device), labels.to(device)
                        optimizer.zero_grad()
                        outputs = model(images)
                        loss = criterion(outputs, labels)
                        loss.backward()
                        optimizer.step()

                # Eval
                model.eval()
                all_preds, all_labels = [], []
                with torch.no_grad():
                    for images, labels in val_loader:
                        images = images.to(device)
                        outputs = model(images)
                        _, preds = torch.max(outputs, 1)
                        all_preds.extend(preds.cpu().numpy())
                        all_labels.extend(labels.numpy())

                acc = accuracy_score(all_labels, all_preds)
                fold_scores.append(acc)

            mean_acc = np.mean(fold_scores)
            results.append({
                'Conv1': conv1_out,
                'Conv2': conv2_out,
                'Dropout': dropout,
                'LR': lr,
                'Val Accuracy': mean_acc
            })

            print(f"Conv({conv1_out}, {conv2_out}), Dropout {dropout}, LR {lr} --> Val Acc: {mean_acc:.4f}")

            if mean_acc > best_score:
                best_score = mean_acc
                best_params = (conv1_out, conv2_out, dropout, lr)

# -----------------------------
# VISUALIZE & FINAL TRAIN
# -----------------------------

results_df = pd.DataFrame(results)
plt.figure(figsize=(10, 6))
sns.barplot(data=results_df, x="LR", y="Val Accuracy", hue="Dropout")
plt.title("CNN Validation Accuracy by Dropout and Learning Rate")
plt.grid(True)
plt.show()

print(f"\nBest Parameters: Conv({best_params[0]}, {best_params[1]}), Dropout {best_params[2]}, LR {best_params[3]}")
print(f"Best CV Accuracy: {best_score:.4f}")
